<h1>Task 1 - DL</h1>

<h3>Step 1: Data Preprocessing</h3>

In [11]:
import os
import cv2

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [12]:
class ArtifactDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir)]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        file_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, file_name)
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Extract the label from the filename
        label = int(file_name.split('_')[-1].split('.')[0])

        if self.transform:
            image = self.transform(image=image)['image']

        return image, label


def get_transforms(phase='train'):
    if phase == 'train':
        return A.Compose([
            A.Resize(224, 224),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.Normalize(),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(),
            ToTensorV2()
        ])

In [26]:
train_dir = "dataset/train"
val_dir = "dataset/test"

train_dataset = ArtifactDataset(train_dir, transform=get_transforms('train'))
val_dataset = ArtifactDataset(val_dir, transform=get_transforms('test'))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

<h3>Step 2: Model Initialization and Fine-Tuning</h3>

In [17]:
from efficientnet_pytorch import EfficientNet
import torch
from torchvision import models
import torch.nn as nn

In [19]:
# For using resnet 
# model = models.resnet18(weights='DEFAULT')
# Load the ResNet-18 model architecture from torchvision
# model.fc = nn.Linear(model.fc.in_features, 2)
# Replace the final fully-connected layer with one matching the number of classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 2) #for binary classification
    
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


<h3>Step 3: Model Training and Validation</h3>

In [22]:
from tqdm import tqdm
from sklearn.metrics import f1_score

In [23]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for images, labels in tqdm(dataloader, desc='Train'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='micro')
    return running_loss / len(dataloader), f1



In [24]:
def validate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validate'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='micro')
    return val_loss / len(dataloader), f1

<h3>Step 4: Model Training and Validation</h3>

In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

best_f1 = 0.0

train_f1_scores = []
val_f1_scores = []

epochs = 10
for epoch in range(epochs):
    print(f"\n🔁 Epoch {epoch + 1}/{epochs}")

    train_loss, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_f1 = validate(model, val_loader, criterion, device)

    train_f1_scores.append(train_f1)
    val_f1_scores.append(val_f1)

    print(f"📈 Train Loss: {train_loss:.4f} | F1: {train_f1:.4f}")
    print(f"🧪 Val Loss:   {val_loss:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'efficientnet_best_model.pt')
        print("✅ Model saved!")

print(f"\n🏁 Best Val F1: {best_f1:.4f}")


🔁 Epoch 1/10


Train:   0%|          | 0/57 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(1, epochs + 1), train_f1_scores, label='Train F1', color='blue')
plt.plot(range(1, epochs + 1), val_f1_scores, label='Val F1', color='red')
plt.xlabel('Epochs')
plt.ylabel('F1 Score (Micro)')
plt.title('Training and Validation F1 Score over Epochs')
plt.legend()
plt.show()